<a href="https://colab.research.google.com/github/vasanthk77/my_projects/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
a=pd.read_csv("sentiment_analysis.csv")
df=pd.DataFrame(a)
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [5]:
## here o represents positive and 1 represents neagtive
## declaring independent and dependent features
X=df.drop(['label','id'],axis='columns')
y=df.label

In [16]:
## 3
## data cleaning using NLP techinques fot text
import nltk
from nltk.stem import PorterStemmer as ps
ps=PorterStemmer()
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
messages=X.copy()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
## applying the stemming techinque here
corpus=[]
for i in range(0,len(messages)):
  new=re.sub('[^a-zA-Z]',' ',messages['tweet'][i])
  new=new.lower()
  new=new.split()

  new=[ps.stem(word) for word in new if not word in stopwords.words('english')]
  new=' '.join(new)
  corpus.append(new)

In [19]:
corpus

['fingerprint pregnanc test http goo gl h mfqv android app beauti cute health iger iphoneonli iphonesia iphon',
 'final transpar silicon case thank uncl yay soni xperia sonyexperia http instagram com p yget jc jm',
 'love would go talk makememori unplug relax iphon smartphon wifi connect http fb n lsupcu',
 'wire know georg made way iphon cute daventri home http instagr p li uj k',
 'amaz servic appl even talk question unless pay stupid support',
 'iphon softwar updat fuck phone big time stupid iphon',
 'happi us instap instadaili us soni xperia xperiaz http instagram com p z qgfwlvj',
 'new type c charger cabl uk http www ebay co uk itm bay amazon etsi new year rob cross tobi young evemun mcmafia taylor spectr newyear start recip technolog samsunggalaxi iphonex pic twitter com pjiwq wtc',
 'bout go shop listen music iphon justm music likeforlik followforfollow http instagr p vj bg tlql',
 'photo fun selfi pool water soni camera picoftheday sun instagood boy cute outdoor http tmblr co 

In [22]:
## applying one_hot encoding for corpus data
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot
vocab_size=5000
one_hot=[one_hot(word,vocab_size) for word in corpus]

In [24]:
one_hot[0]  ## here words reprersents the value with respective vocab_size of 5000

[3728,
 1622,
 2891,
 1766,
 1308,
 15,
 125,
 345,
 2945,
 4314,
 61,
 4123,
 3798,
 3333,
 3527,
 2685,
 2487]

In [28]:
## applying padding sequence for making sure that each doc should have same size
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=15
embedding_layer=pad_sequences(one_hot,maxlen=sent_length,padding='pre')
embedding_layer[5]

array([   0,    0,    0,    0,    0,    0, 2487, 3484,  622, 3235, 2418,
       2596,   49, 2700, 2487], dtype=int32)

In [30]:
## import neccessary things for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding


In [33]:
## Model creation
model=Sequential()
embedding_features=100
model.add(Embedding(vocab_size,embedding_features,input_length=sent_length))
model.add(LSTM(300))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           500000    
                                                                 
 lstm (LSTM)                 (None, 300)               481200    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 981501 (3.74 MB)
Trainable params: 981501 (3.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [38]:
## convert embedded_layers list to arrays
import numpy as np
X=np.array(embedding_layer)
y=np.array(y)

In [39]:
## train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [41]:
## Model checking
model.fit(X_train,y_train,validation_split=0.3,epochs=15)

Epoch 1/15
122/122 [==============================] - 16s 104ms/step - loss: 0.3288 - accuracy: 0.8582 - val_loss: 0.2717 - val_accuracy: 0.8804
Epoch 2/15
122/122 [==============================] - 12s 99ms/step - loss: 0.1787 - accuracy: 0.9304 - val_loss: 0.3606 - val_accuracy: 0.8636
Epoch 3/15
122/122 [==============================] - 12s 99ms/step - loss: 0.1186 - accuracy: 0.9541 - val_loss: 0.3473 - val_accuracy: 0.8636
Epoch 4/15
122/122 [==============================] - 12s 99ms/step - loss: 0.0734 - accuracy: 0.9776 - val_loss: 0.5196 - val_accuracy: 0.8462
Epoch 5/15
122/122 [==============================] - 13s 106ms/step - loss: 0.0486 - accuracy: 0.9840 - val_loss: 0.6160 - val_accuracy: 0.8480
Epoch 6/15
122/122 [==============================] - 14s 114ms/step - loss: 0.0312 - accuracy: 0.9897 - val_loss: 0.7179 - val_accuracy: 0.8419
Epoch 7/15
122/122 [==============================] - 12s 95ms/step - loss: 0.0182 - accuracy: 0.9946 - val_loss: 0.8839 - val_accura

In [43]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred>0.5,1,0)

75/75 [==============================] - 2s 23ms/step


In [44]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)
cm=confusion_matrix(y_test,y_pred)
print(cm)
cr=classification_report(y_test,y_pred)
print(cr)

0.8354377104377104
[[1564  171]
 [ 220  421]]
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1735
           1       0.71      0.66      0.68       641

    accuracy                           0.84      2376
   macro avg       0.79      0.78      0.79      2376
weighted avg       0.83      0.84      0.83      2376



# **Applying :ANN**

In [45]:
##input layer
model.add(Dense(units=11,activation='relu'))
##  adding the first hidden layer
model.add(Dense(units=7,activation='relu'))
##  adding the second hidden layer
model.add(Dense(units=7,activation='relu'))
## output layer
model.add(Dense(units=1,activation='sigmoid'))



In [46]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [48]:
model=model.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=10)

Epoch 1/10
372/372 [==============================] - 29s 67ms/step - loss: 0.3501 - accuracy: 0.9830 - val_loss: 0.5017 - val_accuracy: 0.8377
Epoch 2/10
372/372 [==============================] - 27s 72ms/step - loss: 0.1146 - accuracy: 0.9709 - val_loss: 0.4929 - val_accuracy: 0.8601
Epoch 3/10
372/372 [==============================] - 24s 66ms/step - loss: 0.0837 - accuracy: 0.9801 - val_loss: 0.5950 - val_accuracy: 0.8459
Epoch 4/10
372/372 [==============================] - 25s 67ms/step - loss: 0.0742 - accuracy: 0.9828 - val_loss: 0.5008 - val_accuracy: 0.8607
Epoch 5/10
372/372 [==============================] - 25s 66ms/step - loss: 0.0471 - accuracy: 0.9876 - val_loss: 0.5195 - val_accuracy: 0.8563
Epoch 6/10
372/372 [==============================] - 24s 65ms/step - loss: 0.0225 - accuracy: 0.9949 - val_loss: 0.6699 - val_accuracy: 0.8514
Epoch 7/10
372/372 [==============================] - 23s 63ms/step - loss: 0.0103 - accuracy: 0.9981 - val_loss: 0.7688 - val_accuracy: